In [40]:
from utils.image_shingle import ImageShingle
import os
import pandas as pd
import json
import utils.utils as utils

In [41]:
CRAWL_NAME = 'clickstream-multiple-sc'

25


In [ ]:
CRAWL_PATH = f'crawls/{CRAWL_NAME}/'
PATHS = []
with open(f"crawls/{CRAWL_NAME}/results.json") as results:
    results = json.load(results)
    for path in results:
        if results[path]["crawl_failure"] is False:
            PATHS.append(path)

print(len(PATHS))

In [62]:
def compare_clickstreams(baseline: str, comparison: str, name: str) -> pd.DataFrame:
    rows_list = []

    for i, path in enumerate(PATHS):
        print(f"Analyzing site {i+1}/{len(PATHS)}")
        clickstreams = utils.get_directories(path)

        sample_size = 0

        total_clickstreams = 0
        website_sum = 0

        for clickstream in clickstreams:

            total_actions = 0
            clickstream_sum = 0

            for _ in range(10):
                baseline_base_path = f"{clickstream}/{baseline}-{total_actions}"
                comparison_path = f"{clickstream}/{comparison}-{total_actions}.png"
                
                if os.path.isfile(comparison_path):
                    comparison_shingle = ImageShingle(comparison_path)

                    max_similarity = 0
                    for i in range(1, 11):
                        baseline_path = f"{baseline_base_path}-{i}.png"                        
                        if os.path.isfile(baseline_path):
                            baseline_shingle = ImageShingle(baseline_path)
                            max_similarity = max(max_similarity, baseline_shingle.compare(comparison_shingle))

                    clickstream_sum += max_similarity
                    total_actions += 1

            sample_size += total_actions

            if total_actions != 0:
                clickstream_similarity = clickstream_sum / total_actions
                website_sum += clickstream_similarity

                total_clickstreams += 1
        
        if total_clickstreams != 0:
            website_similarity = website_sum / total_clickstreams
            website_difference = 1 - website_similarity

            website = os.path.basename(os.path.normpath(path))
            rows_list.append({
                "website": website,
                f"difference_{name}": website_difference,
                f"sample_size_{name}": sample_size
            })

    return pd.DataFrame(rows_list)

In [57]:
def compare_with_control() -> pd.DataFrame:
    rows_list = []

    for i, path in enumerate(PATHS):
        print(f"{i+1}/{len(PATHS)} completed")

        clickstreams = utils.get_directories(path)

        sample_size = 0

        total_clickstreams = 0
        website_sum = 0

        for clickstream in clickstreams:

            total_actions = 0
            clickstream_sum = 0

            for _ in range(10):
                baseline_base_path = f"{clickstream}/baseline-{total_actions}"
                control_path = f"{clickstream}/control-{total_actions}.png"
                experimental_path = f"{clickstream}/experimental-{total_actions}.png"
                
                if os.path.isfile(control_path) and os.path.isfile(experimental_path):
                    baseline_shingles = []
                    for i in range(1, 11):
                        baseline_path = f"{baseline_base_path}-{i}.png"
                        if os.path.isfile(baseline_path):
                            baseline_shingles.append(ImageShingle(baseline_path))

                    control_shingle = ImageShingle(control_path)
                    experimental_shingle = ImageShingle(experimental_path)

                    try:
                        clickstream_sum += ImageShingle.compare_with_controls(baseline_shingles, control_shingle, experimental_shingle)
                    except:
                        print(baseline_shingles)
                    total_actions += 1

            sample_size += total_actions

            if total_actions != 0:
                clickstream_similarity = clickstream_sum / total_actions
                website_sum += clickstream_similarity

                total_clickstreams += 1
        
        if total_clickstreams != 0:
            website_similarity = website_sum / total_clickstreams
            website_difference = 1 - website_similarity

            website = os.path.basename(os.path.normpath(path))
            rows_list.append({
                "website": website,
                f"difference": website_difference,
                f"sample_size": sample_size
            })

    return pd.DataFrame(rows_list)

In [63]:
# Two comparison algorithm
control = compare_clickstreams("baseline", "control", "control")
experimental = compare_clickstreams("baseline", "experimental", "remove_cookies")
merged_df = control.merge(experimental, on=["website"])

# One comparison algorithm
control_comparison = compare_with_control()
merged_df = merged_df.merge(control_comparison, on=["website"])

1/25 completed
2/25 completed
3/25 completed
4/25 completed
5/25 completed
6/25 completed
7/25 completed
8/25 completed
9/25 completed
10/25 completed
11/25 completed
12/25 completed
13/25 completed
14/25 completed
15/25 completed
16/25 completed
17/25 completed
18/25 completed
19/25 completed
20/25 completed
21/25 completed
22/25 completed
23/25 completed
24/25 completed
25/25 completed
1/25 completed
2/25 completed
3/25 completed
4/25 completed
5/25 completed
6/25 completed
7/25 completed
8/25 completed
9/25 completed
10/25 completed
11/25 completed
12/25 completed
13/25 completed
14/25 completed
15/25 completed
16/25 completed
17/25 completed
18/25 completed
19/25 completed
20/25 completed
21/25 completed
22/25 completed
23/25 completed
24/25 completed
25/25 completed


In [75]:
merged_df.to_csv(f'analysis/{CRAWL_NAME}-two-comparisons.csv', index=False)